<a href="https://colab.research.google.com/github/AynaAraujo/Aprendendo_Redes_Neurais/blob/main/Introducao_Pytorch/Arquitetura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definindo uma Arquitetura no PyTorch



**Carregando dados**

Exemplo de dados não-linearmente separáveis (distribuição em lua): <br>
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html

In [ ]:
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt

#Definindo pontos e classes
X1, Y1 = make_moons(n_samples=300, noise=0.2)
#Plota graph
plt.scatter(X1[:, 0], X1[:, 1], marker='o',
 c=Y1, s=25, edgecolor='k')

## nn.Sequential

O módulo ```nn.Sequential``` é um container onde se pode colocar múltiplos módulos. Ao realizar um ```forward``` em um objeto ```Sequential``` ele aplicará sequencialmente os módulos nele contidas para gerar uma saída.

Segue abaixo um exemplo desse módulo contendo 2 camadas ```Linear``` intercaladas por uma função de ativação ```ReLU```.

In [ ]:
import torch
from torch import nn # neural networks

input_size = 2 #qtd de entradas
hidden_size = 8 #É um hiperparâmetro, pois cabe ao programador definir a fim de resolver e otimizar problema
output_size = 1 #classes de saída

net = nn.Sequential(nn.Linear(in_features=input_size, out_features=hidden_size), # hidden (escondida)
                    nn.ReLU(),   # ativação não linear
                    nn.Linear(in_features=hidden_size, out_features=output_size)) # output (saída)
print(net)

Sequential(
  (0): Linear(in_features=2, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


O módulo ```summary``` da biblioteca **```torchsummary```** nos permite visualizar mais informações sobre a nossa rede, como quantidade de parâmetros e o tamanho que cada elemento ocupa na memória.

Para mais informações, acesse: https://github.com/sksq96/pytorch-summary

In [ ]:
from torchsummary import summary
summary(net, input_size=(1, input_size))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

### Forward

In [ ]:
#Fluxo de entrada
print(X1.shape) #qtd de dados
tensor = torch.from_numpy(X1).float() #float32
pred   = net(tensor)
print(pred.size())

(300, 2)
torch.Size([300, 1])


## Classe nn.Module

A forma mais organizada de definir modelos em PyTorch é implementando uma classe nos moldes da classe ```nn.Module```. Para redes pequenas, como as que estamos aprendendo até o momento, sua importância pode não se destacar, mas modelos maiores e com funcionalidades mais complexas, são mais fáceis de implementar e realizar manutenções dessa forma.

Funções obrigatórias do ```nn.Module```.
* ```__init()__```: definição de hiperparâmetros e instância do modelo
* ```forward()```: Fluxo da entrada para produzir uma saída

In [ ]:
class MinhaRede(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):
    #É um classe filha de nn.Module
    super(MinhaRede, self).__init__()

    # Definir a arquitetura
    self.hidden = nn.Linear(input_size, hidden_size)
    self.relu   = nn.ReLU()
    self.output = nn.Linear(hidden_size, output_size)

  def forward(self, X):

    # Gerar uma saída a partir do X
    hidden = self.relu(self.hidden(X))
    output = self.output(hidden)

    return output

Instanciando a rede

In [ ]:
input_size = 2
hidden_size = 8
output_size = 1

net = MinhaRede(input_size,hidden_size, output_size)
print(net)

MinhaRede(
  (hidden): Linear(in_features=2, out_features=8, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
)


### Forward

In [ ]:
print(X1.shape)
tensor = torch.from_numpy(X1).float()
pred   = net(tensor)
print(pred.size())

(300, 2)
torch.Size([300, 1])


## Subindo informações na GPU

Para conseguir executar modelos maiores em tempo hábil, é preciso carregar as informação na GPU para que o processamento seja realizado por ela. Dentre as informações que conhecemos até o momento, duas delas precisam ser carregadas ba GPU:
* Entrada
* Rede

A célula a seguir contém a implementação sugerida para realizar esse processo. <br>
Se certifique que o notebook está na configuração desejada (CPU/GPU). Para isso, basta acessar ```Edit -> Notebook Settings``` e definir o hardware desejado.

In [ ]:
#Verificando se está na GPU
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [ ]:
input_size = 2
hidden_size = 8
output_size = 1

net = MinhaRede(input_size,hidden_size, output_size)
#Jogando net na GPU
net = net.to(device)
print(net)

MinhaRede(
  (hidden): Linear(in_features=2, out_features=8, bias=True)
  (relu): ReLU()
  (output): Linear(in_features=8, out_features=1, bias=True)
)


In [ ]:
#Jogando dados(tensor) na GPU
print(X1.shape)
tensor = torch.from_numpy(X1).float()
tensor = tensor.to(device)
pred   = net(tensor)
print(pred.size())

(300, 2)
torch.Size([300, 1])
